In [151]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## 1. Question 2

In [302]:
q2 = pd.read_csv('Q2resource.csv')
q2['region'] = q2['region'].astype('category')
q2['EITIefficiency'] = ((q2['YR_EITIcompliance'] - q2['YR_EITIcandidacy'])**2 + (q2['YR_EITIcandidacy']-q2['YR_EITIannounce'])**2)**0.5/2


In [ ]:
xl = pd.ExcelFile('QMSSDataDiveQ3_final.xlsx')
petroleum = xl.parse('Petroleum exports')
mineral = xl.parse('Mineral exports')

In [303]:
def getData(var, df):
    depend_lst = []
    for name in df.columns:
        if var in name:
            depend_lst.append(name)
    
    lr_dict = {'EITI':[]
             ,'EITIefficiency':[]
             ,'region':[]
             ,'year':[]
             ,'rq':[]
             ,'cc':[]
             ,'dv':[]
             ,'country':[]
             ,'resource':[]}

    for i, yr_compliance in enumerate(df['YR_EITIcompliance']):
        if not np.isnan(yr_compliance):
            start_year = yr_compliance
            reinstate = df['YR_EITIreinstated'].iloc[i]
            while True:
                if start_year == df['YR_EITIsusp1'].iloc[i]:
                    start_year += 1
                    continue
                elif start_year > float(df['YR_EITIsusp1'].iloc[i]) and reinstate == 0:
                    start_year += 1
                    continue
                second_year = var + str(int(start_year+1))
                if second_year in depend_lst:
                    lr_dict['EITI'].append(df['EITI'].iloc[i])
                    lr_dict['region'].append(df['region'].iloc[i])
                    lr_dict['EITIefficiency'].append(df['EITIefficiency'].iloc[i])
                    lr_dict['year'].append(start_year)
                    yr_rq = 'rq_rank_' + str(int(start_year))
                    yr_cc = 'cc_rank_' + str(int(start_year))
                    lr_dict['rq'].append(df[yr_rq].iloc[i])
                    lr_dict['cc'].append(df[yr_cc].iloc[i])
                    first_year = var + str(int(start_year))
                    lr_dict['dv'].append(df[second_year].iloc[i])
                    lr_dict['country'].append(df['country'].iloc[i])
                    lr_dict['resource'].append(df['resource'].iloc[i])
                    start_year += 1
                else:
                    break
    lr_df = pd.DataFrame.from_dict(lr_dict)
    lr_df.to_csv('lr_df_%s.csv' % var, index = False)
    return lr_df

In [292]:
lr_df = getData('yrgdp_', q2)
lr_df = pd.merge(lr_df, petroleum[petroleum['trade_flow'] == 'Export'], how = 'left', left_on = ['country', 'year'], right_on = ['country', 'year'])
lr_df = lr_df.drop(['trade_flow', 'iso', 'commodity', 'unit'], 1)
lr_df.columns = ['EITI', 'EITIefficiency', 'cc', 'country', 'dv', 'region', 'rq', 'year',
       'petroleum_quantity', 'petroleum_value']
lr_df = pd.merge(lr_df, mineral[mineral['trade_flow'] == 'Export'], how = 'left', left_on = ['country', 'year'], right_on = ['country', 'year'])
lr_df = lr_df.drop(['trade_flow', 'iso', 'commodity', 'unit', 'quantity'], 1)
lr_df.columns = ['EITI', 'EITIefficiency', 'cc', 'country', 'dv', 'region', 'rq', 'year',
       'petroleum_quantity', 'petroleum_value', 'mineral_value']
lr_df.to_csv('yrgdp_with_q3.csv')

In [293]:
lr_df['EITIefficiency'].describe()

count    75.000000
mean      2.073947
std       0.800387
min       0.707107
25%       1.581139
50%       2.061553
75%       2.500000
max       3.640055
Name: EITIefficiency, dtype: float64

In [304]:
getData('yrgdp_', q2)
getData('yrpctgdp_', q2)
getData('cc_rank_', q2)

,EITI,EITIefficiency,cc,country,dv,region,resource,rq,year
0,1,3.162278,9.95,Cameroon,10.58,Africa,2,18.96,2013.0
1,1,3.162278,10.58,Cameroon,12.98,Africa,2,19.71,2014.0
2,1,3.162278,12.98,Cameroon,11.06,Africa,2,16.83,2015.0
3,1,2.500000,6.25,Chad,5.29,Africa,2,10.58,2014.0
4,1,2.500000,5.29,Chad,4.81,Africa,2,9.13,2015.0
5,1,2.500000,57.62,Ghana,57.35,Africa,2,54.55,2010.0
6,1,2.500000,57.35,Ghana,54.98,Africa,2,55.45,2011.0
7,1,2.500000,54.98,Ghana,55.45,Africa,2,56.40,2012.0
8,1,2.500000,55.45,Ghana,52.40,Africa,2,55.45,2013.0
9,1,2.500000,52.40,Ghana,52.88,Africa,2,53.37,2014.0


In [125]:
def transform(var, df):
    col_name = df.columns
    need_lst = ['country']
    year_lst = []
    for name in col_name:
        if var in name:
            str_split = name.split('_')
            year_lst.append(str_split[len(str_split)-1])
            need_lst.append(name)
    need_df = df[need_lst]
    need_df.columns = year_lst + ['country']
    need_df.index = need_df['country']
    need_df = pd.DataFrame.transpose(need_df.drop('country', 1))
    return need_df

In [ ]:
gdp_df = transform('yrgdp_', q2)
pctgdp_df = transform('yrpctgdp_', q2)
rq_est_df = transform('rq_est_', q2)

gdp_df.to_csv('q2_gdp.csv')
pctgdp_df.to_csv('q2_pctgdp.csv')

In [128]:
def find_ts(var, df):
    col_name = df.columns
    need_lst = []
    year_lst = []
    for name in col_name:
        if var in name:
            str_split = name.split('_')
            year_lst.append(str_split[len(str_split)-1])
            need_lst.append(name)
    need_df = df[need_lst]
    need_df.columns = year_lst
    return need_df

In [146]:
var_lst = q2.columns
drop_lst = var_lst[0:20]
numeric_df = q2.drop(drop_lst, 1)
numeric_df['region'] = q2['region']
group_df = numeric_df.groupby('region').mean()

In [148]:
find_ts('yrgdp_', group_df).dropna(1).to_csv('region_gdp.csv')
find_ts('yrpctgdp_', group_df).dropna(1).to_csv('region_pctgdp.csv')
find_ts('cc_rank_', group_df).dropna(1).to_csv('region_cc.csv')

## 2. Question 1

In [231]:
q1 = pd.read_csv('Q1_CompanyIndicators_FINAL-OilGas.csv')
q1['start'] = range(len(q1))

In [232]:
judge_df = q1.isnull()
for i in range(len(q1)):
    if not judge_df['Norway (Bors/Axess)'].iloc[i]:
        if 'Yes' in q1['Norway (Bors/Axess)'].iloc[i]:
            q1.set_value(i, 'start', 2014)
    elif not judge_df['EU Transparency Directive'].iloc[i]:
        if 'Yes' in q1['EU Transparency Directive'].iloc[i]:
            q1.set_value(i, 'start', 2015)
    elif not judge_df['Toronto (TSX/Venture)'].iloc[i]:
        if 'Yes' in q1['Toronto (TSX/Venture)'].iloc[i]:
            q1.set_value(i, 'start', 2016)
    else:
        q1.set_value(i, 'start', 0)
q1.to_csv('q1-oilgas.csv')

In [201]:
col_name = q1.columns
select_df = q1.drop(col_name[0:14], 1)
select_df['Transparent'] = q1['Transparent'].astype('category')
select_df.drop('start', 1).groupby('Transparent').mean().to_csv('oilgas_grouped.csv')

In [243]:
def Data(var, df):
    depend_lst = []
    for name in df.columns:
        if var in name:
            depend_lst.append(name)
    
    lr_dict = {'Transparent':[]
              ,'Year':[]
              ,'Season':[]
              ,'dv':[]
              ,'cap':[]}

    for i, start_yr in enumerate(df['start']):
        flag_yr = 2014
        flag_season = 1
        while True:
            if flag_season == 4:
                next_data = var + ' ' + 'Q1' + ' ' + str(flag_yr+1)
            else:
                next_data = var + ' ' + 'Q' + str(flag_season+1) + ' ' + str(flag_yr)
            print(next_data)
            if next_data in depend_lst and not df[next_data].isnull().iloc[i]:
                if start_yr >= flag_yr:
                    lr_dict['Transparent'].append(1)
                else:
                    lr_dict['Transparent'].append(0)
                lr_dict['Year'].append(flag_yr)
                lr_dict['Season'].append(flag_season)
                lr_dict['dv'].append(df[next_data].iloc[i])
                lr_dict['cap'].append(df['2014 Market Cap ($Bln)'].iloc[i])
                if flag_season >= 4:
                    flag_season = 1
                    flag_yr += 1
                else:
                    flag_season += 1
            elif next_data in depend_lst:
                if flag_season >= 4:
                    flag_yr += 1
                    flag_season = 1
                else:
                    flag_season += 1
                continue
            else:
                break
    lr_df = pd.DataFrame.from_dict(lr_dict)
    lr_df.to_csv('lr_df_%s_oilgas.csv' % var, index = False)

In [247]:
Data('PE', q1)

PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014
PE Q3 2014
PE Q4 2014
PE Q1 2015
PE Q2 2015
PE Q3 2015
PE Q4 2015
PE Q1 2016
PE Q2 2016
PE Q3 2016
PE Q4 2016
PE Q1 2017
PE Q2 2017
PE Q3 2017
PE Q4 2017
PE Q2 2014